In [8]:
type LinkedListNode{T}
    entry::T
    next::Nullable{LinkedListNode{T}}
end

# Create a linked list node pointing to nothing
function LinkedListNode{T}(entry::T) where T
    return LinkedListNode{T}(entry, Nullable{T}())
end

type LinkedList{T}
    first::Nullable{LinkedListNode{T}}
    last::Nullable{LinkedListNode{T}}
end

# Create an empty linked list
function LinkedList{T}() where T
    return LinkedList{T}(Nullable{T}(), Nullable{T}())
end

import Base.start
function start(ll::LinkedList{T}) where T
    return ll.first
end

import Base.next
function next(ll::LinkedList{T}, state::Nullable{LinkedListNode{T}}) where T
    return (state.entry, state.next)
end

import Base.done
function done(ll::LinkedList{T}, state::Nullable{LinkedListNode{T}}) where T
    return isnull(state)
end

import Base.push!
function push!(ll::LinkedList{T}, items::Vararg{T,N} where N) where T
    for item in items
        newnode = LinkedListNode{T}(item)
        if isnull(ll.first)
            ll.first = newnode
            ll.last = newnode
        else
            get(ll.last).next = newnode
            ll.last = newnode
        end
    end
    return ll
end

import Base.shift!
function shift!(ll::LinkedList{T}) where T
    firstentry = get(ll.first).entry
    if isnull(get(ll.first).next)
        ll = LinkedList{T}()
    else
        ll.first = get(ll.first).next
    end
    return firstentry
end

LoadError: [91minvalid redefinition of constant LinkedListNode[39m

In [33]:
# Performs a breadth-first search to find a state / occupation vector satisfying
# constraints (passed as a boolean function of P, Q of current state).
# Pvec and Qvec are vectors with entry i corresponding to P or Q of state i
# translist is an array with +1 / -1 elements corresponding to state changes after each transition
function find_satisfier(constraints, startvec, Pvec, Qvec, translist)
    currvec = startvec
    visitedset = Set{Array{Int,1}}()
    nextset = Set{Array{Int,1}}()
    nextll = LinkedList{Array{Int,1}}()
    
    while !constraints(vecdot(currvec, Pvec), vecdot(currvec, Qvec))
        union!(visitedset, [currvec])
        # Apply all transitions to currvec to get next vecs to search
        for tlrow in 1:size(translist,1)
            vec = currvec + translist[tlrow,:]
            if all(vec .>= 0) && !in(vec, visitedset) && !in(vec, nextset)
                # If this is a new + valid occupation vector add it to be visited later
                union!(nextset, [vec])
                push!(nextll, vec)
            end
        end
        
        if isempty(nextset)
            error("Solution not found.")
        else
            currvec = shift!(nextll)
            setdiff!(nextset, [currvec])
        end
    end
    
    return currvec
end

# Computes a starting vector for DER_graph_explorer from the DERtable and
# statetable (both coming from an AQL DERNetworkInState)
function getstartvec(DERs, states)
    stateidcol = findfirst(map(x -> x == "id", states[1,:]))
    instatecol = findfirst(map(x -> x == "in_state", DERs[1,:]))
    stateids::Array{Int,1} = states[2:end,stateidcol]
    DERstateids::Array{Int,1} = DERs[2:end,instatecol]
    numinstate = stateid -> count(x -> x == stateid, DERstateids)
    startvec = map(numinstate, stateids)
end

# Converts an occupation vector into a nice, readable string
function getstringspecifier(vec, DERtypes, states)
    result = ""
    stateidcol = findfirst(map(x -> x == "id", states[1,:]))
    statenamecol = findfirst(map(x -> x == "name", states[1,:]))
    fortypecol = findfirst(map(x -> x == "for_type", states[1,:]))
    typeidcol = findfirst(map(x -> x == "id", DERtypes[1,:]))
    typenamecol = findfirst(map(x -> x == "name", DERtypes[1,:]))
    for i in 2:size(DERtypes,1)
        result *= "$(DERtypes[i,typenamecol]):"
        stateindices = find(x -> x == DERtypes[i,typeidcol], states[:,fortypecol])
        for j in stateindices
            result *= " $(vec[j-1]) $(states[j,statenamecol])"
        end
        if i != size(DERtypes,1)
            result *= "\n"
        end
    end
    return result
end

getstringspecifier (generic function with 1 method)

In [42]:
# Import files from AQL containing info on behavior, number of DERs
folderpath = "C:/Users/jsn1/Documents/AQL/DER/example"
DERs = readcsv(folderpath * "/DER.csv")
DERtypes = readcsv(folderpath * "/DERType.csv")
states = readcsv(folderpath * "/state.csv")
transitions = readcsv(folderpath * "/transition.csv")

# Get starting state and desired constraints
# Starting state specified as occupation vector (vector with element i = 
# number of DERs in state i, where order is same as in state.csv ???)
# Constraints are specified as Boolean-valued function of P and Q (f: R^2 -> B)
startvec = getstartvec(DERs, states)

constraints = (P,Q) -> (Q - P >= 25)

# Generate 'list' (actually a matrix) of all possible transitions
# Rows correspond to transitions, columns correspond to states
# Each row has all zeros except for a -1 (from state) and a +1 (to state)
stateidcol = findfirst(map(x -> x == "id", states[1,:]))
statetoindex = Dict{Int,Int}(map(i -> (states[i,stateidcol], i), 2:size(states,1)))
fromstatecol = findfirst(map(x -> x == "from_state", transitions[1,:]))
tostatecol = findfirst(map(x -> x == "to_state", transitions[1,:]))
translist = zeros(Int,size(transitions,1)-1, size(states,1)-1)
for i = 2:size(transitions,1)
    translist[i-1, statetoindex[transitions[i,fromstatecol]]-1] = -1
    translist[i-1, statetoindex[transitions[i,tostatecol]]-1] = 1
end

# Get P's and Q's for each state (in same order as state.csv)
Pcol = findfirst(map(x -> x == "P", states[1,:]))
Qcol = findfirst(map(x -> x == "Q", states[1,:]))
Pvec = states[2:end,Pcol]
Qvec = states[2:end,Qcol]

# Given a starting vertex / "current state", do a breadth-first search on
# the product until you reach a vertex / state meeting conditions.
# Return the occupation vector for the found state
# From here it should be obvious how to reach the given state
result = find_satisfier(constraints, startvec, Pvec, Qvec, translist)

# Print results
println(getstringspecifier(result, DERtypes, states))

HVAC: 0 Low 10 Off 0 High
PV: 0 Off 2 High 9 Low
Battery: 5 Off 0 Discharging 0 Charging


In [5]:
(x, y) = (1, 2)
x

1

In [4]:
println("hey \nyou")

hey 
you
